# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Save the dataframe as a CSV file for offline use
loans_df.to_csv("../data/student_loans.csv", index=False)

In [4]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [5]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
y[:5]

array([0, 0, 0, 1, 0], dtype=int64)

In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking")

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Scale the features training and testing datasets
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [10]:
# Define the the number of inputs (features) to the model
num_features = X.shape[1]

# Review the number of features
num_features

11

In [11]:
# Define the number of hidden nodes for the first hidden layer
hl_nodes_1 = 6

# Define the number of hidden nodes for the second hidden layer
hl_nodes_2 = 3

# Define the number of neurons in the output layer
ol_neurons = 1

In [12]:
# Create the Sequential model instance
nn_model = Sequential()

# Add the first hidden layer
nn_model.add(Dense(units=hl_nodes_1, activation="relu", input_shape=(num_features,)))

# Add the second hidden layer
nn_model.add(Dense(units=hl_nodes_2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=ol_neurons, activation="sigmoid"))

In [13]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [14]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Fit the model using 50 epochs and the training data
fit_response = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50


38/38 [==============================] - 2s 2ms/step - loss: 0.7559 - accuracy: 0.4621
Epoch 2/50
38/38 [==============================] - 0s 1ms/step - loss: 0.7085 - accuracy: 0.4987
Epoch 3/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6729 - accuracy: 0.5371
Epoch 4/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6416 - accuracy: 0.5671
Epoch 5/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6157 - accuracy: 0.5947
Epoch 6/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5968 - accuracy: 0.6272
Epoch 7/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5795 - accuracy: 0.6997
Epoch 8/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5637 - accuracy: 0.7248
Epoch 9/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5502 - accuracy: 0.7331
Epoch 10/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5413 - accuracy: 0.7356
Epoch 11/50
38/38

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [16]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5220 - accuracy: 0.7350 - 293ms/epoch - 23ms/step
Loss: 0.5219992995262146, Accuracy: 0.7350000143051147


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [20]:
# Set the model's file path
model_file_path = "../models/student_loans.keras"

# Export your model to a keras file
nn_model.save(model_file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [21]:
# Set the model's file path
model_file_path = "../models/student_loans.keras"

# Load the model to a new object
saved_nn_model = tf.keras.models.load_model(model_file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [22]:
# Make predictions with the test data
y_pred = saved_nn_model.predict(X_test_scaled)

# Display a sample of the predictions
y_pred[:5]

13/13 [==============================] - 0s 2ms/step


array([[0.6514709 ],
       [0.26326925],
       [0.74432796],
       [0.6217857 ],
       [0.98488754]], dtype=float32)

In [26]:
# Save the predictions to a DataFrame and round the predictions to binary results
y_pred_df = pd.DataFrame(y_pred, columns=["predictions"])
y_pred_df["predictions"] = y_pred_df["predictions"].apply(lambda x: 1 if x >= 0.5 else 0)
y_pred_df

,predictions
0,1
1,0
2,1
3,1
4,1
...,...
395,1
396,0
397,1
398,0


### Step 4: Display a classification report with the y test data and predictions

In [28]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, y_pred_df))

              precision    recall  f1-score   support

           0       0.69      0.79      0.74       188
           1       0.79      0.68      0.73       212

    accuracy                           0.73       400
   macro avg       0.74      0.74      0.73       400
weighted avg       0.74      0.73      0.73       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

* In addition to the financial and academic data used to predict student loan eligibility; I would collect student geogrphic, demographic, preference, career, and earnings data; loan product data; and data on which loans other students have taken in the past. With this type of data, the system would be able to recommend loans that are most likely to be accessible to and accepted by students.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

* With the data above, the system could use a combination of collaborative and content-based filtering. 
  * Collaborative filtering could be used to compare current students to recent students within the same demographic and geographic groups to compile a list of accessible loan options.
  * Content-based filtering could then be used to further refine the list of loan recommendations by comparing students with similar preferences, careers, and potential earnings.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

* First would be data privacy and security concerns. The system would be handling sensative personal, financial, and academic data which has legal and regulatory implications if not stored, secured, and used appropriately.
* Second would be equity conerns. The system would have to be modeled to reduce biases for and against races, genders, and nationalities. Thus ensuring that the system would operate effectively, be trustworthy, and serve students in a manner that is legally and ethically sound.